**Step 1 - Data Understanding**
- shape
- columns
- head & tail
- info
- dtypes
- describe
    
**Step 2 - Data Preparation**
- Droping irrelative Columns and Rows
- Identifying duplicated columns
- Renaming columns
- Feature creation
    
**Step 3 - Feature Understanding** (univariate analysis)
- Plotting features distributions
    - Histogram
    - KDE
    - Box plot
    
**Step 4 - Feature Relationship**
- Scatter plot
- Heatmap correlation
- Pair plot
- Groupby comparisons

### Import required libraries

In [ ]:
import pandas as pd


### Import Dataset

In [ ]:
df = pd.read_excel('Data_Pathrise.xlsx')

## Step 1 - Data understanding

### shape

In [ ]:
df.shape

### columns

In [ ]:
df.columns

### info

In [ ]:
df.info()

### head & tail

In [ ]:
df.head()

In [ ]:
df.tail()

### dtypes

In [ ]:
df.dtypes

### describe

In [ ]:
df.describe()

## Step 2 - Data Preparation

### Droping irrelative Columns and Rows

### Identifying duplicated columns

### Renaming columns

### Feature creation

## Step 3 - Feature Understanding (univariate analysis)

### Plotting features distributions

#### Histogram

#### KDE

#### Box plot

## Step 4 - Feature Relationship

### Scatter plot

### Heatmap correlation

### Pair plot

### Groupby comparisons